# Overview

1. Given a star at REFO of known coordinates, astrometrically solve for the center of the MOSFIRE imaging FoV and calculate the distance and angle on the sky between the two points.
1. Subtract out the angle of the rotator (on the sky) to determine the distance (angle on the sky) and rotation (angle on instrument) between the two points.
1. Given a number of these measurements, fit the relationship between telescope elevation, rotator physical angle, and the 2D separation of the guider and instrument FoV.
1. Translate the differences to a motion of the guider (REFO) relative to the main instrument in units of guider pixels.

# Observing Procedure

1. OA: Pick a pointing star near desired EL.
1. OA: Choose rotator position angle (PA) near rotator drive angle 0.
1. OA: Slew to star using PO REF (rotator mode should be PositionAngle).
1. OA: Center star on REF using Ca, Ce adjustments and begin guiding.
1. SA: Note guide star name and coordinates (for insurance).
1. SA: Image field in J-band.
1. OA: Rotate 45 degrees (rotator mode should be PositionAngle).
1. OA: Center star on REF using Ca, Ce adjustments and begin guiding.
1. SA: Note guide star name and coordinates (for insurance).
1. SA: Image field in J-band.
1. Keep repeating steps 7-10 until you have rotated 360 degrees.
1. Then repeat for a new EL.

## Data Table

Raw data: for each image, fill in:
1. file name
1. telescope elevation
1. instrument PA
1. guider RA
1. guider Dec

Loop through all elements of the data table and derive the values in the measurements table.

From the image data, we derive:
1. the Measured Sky Angle: PA (East of North) between guider and instrument FoV.
1. the Physical Angle: The Angle between them after the instrument PA has been subtracted off.  This should be constant other than flexure.
1. the Separation: distance (in arcsec) between the two FoVs.  This should be constant other than flexure.

Relevant MOSFIRE keywords:

`show -s mosfire SKYPA0 SKYPA1 SKYPA2 SKYPA3 FCPA_EL`

MOSFIRE FITS header keywords:

`SKYPA0 SKYPA1 SKYPA2 SKYPA3 FCPA_EL`

In [3]:
from pathlib import Path

import ccdproc
from ccdproc import ImageFileCollection

from astropy.table import QTable
from astropy import coordinates as c
from astropy import units as u
import numpy as np

In [12]:
table_file = Path('~/KeckData/MOSFIRE_GuiderFlexure/ImageResults.txt').expanduser()
t = QTable.read(table_file, format='ascii')

In [13]:
t

Filename,EL,PA,RotAng,GuiderCoord,ImageCoord,Offset Distance,Offset Angle
,deg,deg,deg,,,arcsec,deg
str17,float64,float64,float64,str24,str24,float64,float64
m170707_0490.fits,28.39876332,0.16131778,-85.44270876,16:40:39.95 +36:20:52.88,16:40:40.28 +36:20:50.38,4.640089552243393,302.48397005104687


In [14]:
t['Offset Distance'].unit

Unit("arcsec")